In [1]:
import os
import time
import duckdb
import pandas as pd
from dotenv import load_dotenv

In [2]:
%load_ext sql
conn = duckdb.connect()
%sql conn --alias duckdb

In [3]:
# to receive feedback from SQL queries
%config SqlMagic.feedback = False
%config SqlMagic.displaylimit = None  # None means unlimited
%config SqlMagic.style = "SINGLE_BORDER"  

# some people like to use this
# %config SqlMagic.autopandas = True
# but this will make the notebook slow

displaylimit: Value None will be treated as 0 (no limit)

In [4]:
%%sql 
select 42 as my_answer

my_answer
42


In [6]:
%%sql column_info <<
describe
select * from read_json_auto('./data/goodreads_books.json.gz')

In [7]:
print(column_info)

┌──────────────────────┬────────────────────────────────────────────┬──────┬──────┬─────────┬───────┐
│     column_name      │                column_type                 │ null │ key  │ default │ extra │
├──────────────────────┼────────────────────────────────────────────┼──────┼──────┼─────────┼───────┤
│         isbn         │                  VARCHAR                   │ YES  │ None │   None  │  None │
│  text_reviews_count  │                  VARCHAR                   │ YES  │ None │   None  │  None │
│        series        │                  BIGINT[]                  │ YES  │ None │   None  │  None │
│     country_code     │                  VARCHAR                   │ YES  │ None │   None  │  None │
│    language_code     │                  VARCHAR                   │ YES  │ None │   None  │  None │
│   popular_shelves    │   STRUCT(count BIGINT, "name" VARCHAR)[]   │ YES  │ None │   None  │  None │
│         asin         │                  VARCHAR                   │ YES  │ None 

In [10]:
%%sql book_sample <<
select isbn text_reviews_count, country_code, average_rating,
language_code, link, url
from read_json_auto('./data/goodreads_books.json.gz')
using sample 5;

In [11]:
print(book_sample)

┌────────────────────┬──────────────┬────────────────┬───────────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│ text_reviews_count │ country_code │ average_rating │ language_code │                                                                                  link                                                                                 │                                                                                  url                                                                                  │
├────────────────────┼──────────────┼────────────────┼───────────────┼────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [14]:
%%sql review_info <<
describe
select * from read_json_auto('./data/goodreads_reviews_dedup.json.gz');

In [15]:
print(review_info)

┌──────────────┬─────────────┬──────┬──────┬─────────┬───────┐
│ column_name  │ column_type │ null │ key  │ default │ extra │
├──────────────┼─────────────┼──────┼──────┼─────────┼───────┤
│   user_id    │     UUID    │ YES  │ None │   None  │  None │
│   book_id    │    BIGINT   │ YES  │ None │   None  │  None │
│  review_id   │     UUID    │ YES  │ None │   None  │  None │
│    rating    │    BIGINT   │ YES  │ None │   None  │  None │
│ review_text  │   VARCHAR   │ YES  │ None │   None  │  None │
│  date_added  │   VARCHAR   │ YES  │ None │   None  │  None │
│ date_updated │   VARCHAR   │ YES  │ None │   None  │  None │
│   read_at    │   VARCHAR   │ YES  │ None │   None  │  None │
│  started_at  │   VARCHAR   │ YES  │ None │   None  │  None │
│   n_votes    │    BIGINT   │ YES  │ None │   None  │  None │
│  n_comments  │    BIGINT   │ YES  │ None │   None  │  None │
└──────────────┴─────────────┴──────┴──────┴─────────┴───────┘


In [16]:
%%sql country_count <<
select country_code, count(*) as count
from read_json_auto('./data/goodreads_books.json.gz')
group by country_code

In [17]:
print(country_count)

┌──────────────┬─────────┐
│ country_code │  count  │
├──────────────┼─────────┤
│      US      │ 2360165 │
│              │   490   │
└──────────────┴─────────┘


In [20]:
%%sql language_count <<
select language_code, count(*) as count
from read_json_auto('./data/goodreads_books.json.gz')
group by language_code
order by count desc

In [21]:
print(language_count)

┌───────────────┬─────────┐
│ language_code │  count  │
├───────────────┼─────────┤
│               │ 1060153 │
│      eng      │  708457 │
│     en-US     │  91452  │
│     en-GB     │  58358  │
│      spa      │  54524  │
│      ita      │  50902  │
│      ara      │  42978  │
│      fre      │  32046  │
│      ger      │  30941  │
│      ind      │  27291  │
│      por      │  23452  │
│       nl      │  17497  │
│      tur      │  14238  │
│      per      │  11821  │
│      fin      │  11611  │
│      gre      │  10024  │
│      swe      │   9914  │
│      cze      │   8564  │
│     en-CA     │   7652  │
│      jpn      │   7209  │
│      bul      │   7105  │
│      rus      │   6617  │
│      pol      │   6576  │
│      msa      │   5675  │
│      rum      │   5216  │
│      dan      │   5159  │
│      ben      │   3385  │
│      vie      │   3372  │
│      tha      │   3106  │
│      scr      │   3022  │
│      nor      │   2705  │
│      srp      │   2341  │
│      est      │   